In [171]:
import os
import io
import numpy as np
import random
import re

In [172]:
def unique(arr, dic=None):
    if (dic is None):
        dic = {}
    for el in arr:
        if isinstance(el, list):
            unique(el, dic)
        else:
            if (el not in dic):
                dic[el] = 1
            else:
                dic[el] += 1
    return np.array(dic.keys())

Классификация будет происходить по след формуле:
$$p(c\mid d,\lambda)=\frac
{\exp\sum_i^{n \times k}{\lambda_i}f_i\left(c,d\right )}
{\sum_{\tilde{c}\in C}{\exp\sum_i^{n \times k}{\lambda_i}f_i\left(\tilde{c},d\right )}}$$

In [173]:
def predict(x, weights, feature_patterns, n_classes):
    # начальное приведение
    probas = np.ones(n_classes) * np.log(1.0 / n_classes)

    # считаем условные вероятности
    for xi in x:
        for i in xrange(len(feature_patterns[xi])):
            probas[feature_patterns[xi][i]] += weights[xi][i]

    # далее сглаживаем выходы через softmax
    probas = np.exp(probas / n_classes)
    return probas / np.sum(probas)

Задачу будем решать с помощью максимизации функции правдоподобия
$$\log p(C|D,\lambda)
=\sum_{(c,d)\in(C,D)}\log p(c|d,\lambda)
=\sum_{(c,d)\in(C,D)}\log\frac
{\exp\sum_i^{n \times k}{\lambda_i}f_i\left(c,d\right )}
{\sum_{\tilde{c}\in C}{\exp\sum_i^{n \times k}{\lambda_i}f_i\left(\tilde{c},d\right )}}$$

Соответственно градиент у нас будет в частных производных

$$\frac{\partial\log p(C|D,\lambda)}{\partial\lambda_i}=
\sum_{(c,d)\in(C,D)}{f_i(c,d)}-
\sum_{d\in D}{\sum_{c\in C}{p(c|d,\lambda)f_i(c,d)}}$$

итого:
$$w^{k+1} = w^{k} + \eta_k\frac{\partial}{\partial w_i}(L(j,w) - \frac{C}{N}|w_i|)$$

In [190]:
# L(j,w)
def apply_l1_penalty(i, j, u, weights, q):
    z = weights[i][j]
    if weights[i][j] > 0:
        weights[i][j] =  max(0.0, weights[i][j] - (u + q[i][j]))
    elif weights[i][j] < 0:
        weights[i][j] =  max(0.0, weights[i][j] + (u - q[i][j]))
    q[i][j] = weights[i][j] - z

In [175]:
def fit(X, y, f_count, c_count, batch_size=64, alpha=0.85, max_iter=100, tol=0.00001, l1param=0.1, random_state=None, verbose=1):
    '''
        X - контекст 
        y - цель
        f_count - количество уникальных признаков в контексте
        c_count - количество уникальных целей, по факту искомые классы
        batch_size - размер подвыборки для обучения
        alpha - быстрота обучения
        max_iter - макс кол-во итераций
        tol - порог смещения градиента
        l1param - параметр l1 регуляризации
        random_state - для репродуцирования эксперимента
        verbose - дебаг
    '''
    n_samples = len(X)
    if batch_size is None:
        batch_size = n_samples
    if batch_size > n_samples:
        batch_size = n_samples
    if random_state is not None:
        random.seed(random_state)

#     # определяем сколько у нас уникальных токенов
#     features = unique(X)
#     f_count = features.shape[0]
#     # определяем сколько у нас уникальных классов
#     classes = unique(y)
#     c_count = classes.shape[0]

    # матрица индикаторов(условных признаков)
    feature_patterns = [[] for _ in range(f_count)]
    f_pattern_set = {}
    # матрица весов индикаторов
    weights = [[] for _ in range(f_count)]
    q =  [[] for _ in range(f_count)]
    # инициализация индикаторов
    for i in range(n_samples):
        for xi in X[i]:
            if xi not in f_pattern_set:
                f_pattern_set[xi] = set()
            if y[i] not in f_pattern_set[xi]:
                f_pattern_set[xi].add(y[i])
                weights[xi].append(0.0)
                q[xi].append(0.0)
                feature_patterns[xi].append(y[i])
    prev_logl = 0.
    iter_num = 0
    all_iter = 0
    u = 0.0
    # ограничим сверху max_iter итерациями
    for iter_num in range(max_iter):
        if verbose:
            print 'Start iteration #%d\t' % iter_num,

        logl = 0.
        ncorrect = 0

        # random прохождение существенно улучшает схождение SGD
        r = range(n_samples)
        r = random.sample(r, batch_size)
        iter_sample = 0
        for i in r:
            iter_sample += 1
            if verbose and (batch_size < 20 or iter_sample % (batch_size / 20) == 0):
                print '.',

            all_iter += 1
            eta = alpha ** (all_iter / n_samples)
            # предсказываем вероятности
            probas = predict(X[i], weights, feature_patterns, c_count)

            # смотрим, правильно ли мы предсказали, это нужно только для verbose
            if np.argmax(probas) == y[i]:
                ncorrect += 1
            # считаем "правдоподобие"
            logl += np.log(probas[y[i]])
            
            u += eta * l1param;
            # обновляем веса
            for j in range(len(X[i])):
                conditional_y = feature_patterns[X[i][j]]
                for y_i in xrange(len(conditional_y)):
                    # ожидание
                    expected_ent = 1.0 if conditional_y[y_i] == y[i] else 0.0
                    # реальность
                    max_ent = probas[conditional_y[y_i]]
                    weights[X[i][j]][y_i] -= 1.0 * (max_ent - expected_ent) * eta
                    apply_l1_penalty(X[i][j],y_i,u,weights,q)
        logl /= c_count
        if verbose:
            f = logl
            sum = 0.0
            for val_x in weights:
                for val_y in val_x:
                    sum += abs(val_y)
            f -= l1param * sum;
            print '\tAccuracy: %.5f, Loss: %.8f' % (1.0 * ncorrect / batch_size, f)
        if iter_num > 0:
            if prev_logl > logl:
                break
            if (logl - prev_logl) < tol:
                break
        prev_logl = logl
    print iter_num
    return weights, feature_patterns

In [176]:
# небольшой тест
X = [[0, 1],
     [2, 1],
     [2, 3],
     [2, 1],
     [0, 1],
     [2, 1, 4],
     [2, 3, 4],
     [2, 1, 5],
     [0, 3, 5],
     [0, 1, 5]]

y = [0, 0, 1, 1, 0, 1, 1, 0, 0, 0]
# определяем сколько у нас уникальных токенов
features = unique(X)
f_count = features.shape[0]
# определяем сколько у нас уникальных классов
classes = unique(y)
c_count = classes.shape[0]
weights, patterns = fit(X, y,f_count,c_count, random_state=241,l1param=0.00001)
# print weights
# print patterns

pred = predict([0, 1], weights, patterns,c_count)
print pred

Start iteration #0	. . . . . . . . . . 	Accuracy: 0.70000, Loss: -3.08720890
Start iteration #1	. . . . . . . . . . 	Accuracy: 0.90000, Loss: -2.15663061
Start iteration #2	. . . . . . . . . . 	Accuracy: 0.80000, Loss: -1.81543056
Start iteration #3	. . . . . . . . . . 	Accuracy: 0.80000, Loss: -1.48595419
Start iteration #4	. . . . . . . . . . 	Accuracy: 0.90000, Loss: -1.39364816
Start iteration #5	. . . . . . . . . . 	Accuracy: 0.90000, Loss: -1.27724560
Start iteration #6	. . . . . . . . . . 	Accuracy: 0.80000, Loss: -1.28469326
6
[ 0.9355966  0.0644034]


In [177]:
digits_regex = re.compile('\d')
punc_regex = re.compile('[\%\(\)\-\/\:\;\<\>\«\»\,]')
delim_regex = re.compile('([\.])\s+')

In [178]:
def read_and_tokenize(foldername):
    '''
    метод для считывания текстов из файлов папки
    здесь применяется довольно простая токенизация
    '''

    word_counts = {}
    tokenized_text = []
    for path, subdirs, files in os.walk('data'):
        for name in files:
            filename = os.path.join(path, name)
            with io.open(filename, 'r', encoding='utf-8') as data_file:
                for line in data_file:
                    if len(line) < 50:
                        continue
                    text = digits_regex.sub(u'0', line.lower())
                    text = punc_regex.sub(u'', text)
                    text = delim_regex.sub(r' \1 ', text)
                    for word in text.split():
                        if not word:
                            continue
                        if word not in word_counts:
                            word_counts[word] = 1
                        else:
                            word_counts[word] += 1
                        tokenized_text.append(word)
    word2index = {}
    index2word = []
    i = 0
    filtered_text = []
    for word in tokenized_text:
        if word_counts[word] > 2:
            if word not in word2index:
                word2index[word] = i
                index2word.append(word)
                i += 1
            filtered_text.append(word)


    return filtered_text, word2index, index2word

In [179]:
def generate_train(tokenized_text, word2index,context_len = 4):
    '''
    метод для генерации обучающих данных
    '''
    X = []
    y = []
    for i, y_word in enumerate(tokenized_text):
        x = []
        for j in range(i - context_len, i):
            if (j >= 0):
                x_word = tokenized_text[j]
                x.append(word2index[x_word])
        if (len(x) > 0):
            X.append(x)
            y.append(word2index[y_word])
    return X, y

In [180]:
tokenized_text, word2index, index2word = read_and_tokenize('data')      

In [181]:
unique_words = len(index2word)
print 'all words:', len(tokenized_text)
print 'all unique words:', unique_words

all words: 45825
all unique words: 3872


In [182]:
context_len = 4
X,y = generate_train(tokenized_text, word2index,context_len=context_len)

In [183]:
weights, patterns = fit(X, y,unique_words,unique_words,random_state=241,verbose=1,batch_size=1024, l1param=0.0001,tol=0.000001)

Start iteration #0	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.04395, Loss: -5.60062674
Start iteration #1	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.04883, Loss: -12.27356474
Start iteration #2	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.05469, Loss: -28.91910292
Start iteration #3	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.07520, Loss: -50.19205419
Start iteration #4	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.08691, Loss: -100.62756221
Start iteration #5	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.10156, Loss: -114.63170959
Start iteration #6	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.11230, Loss: -158.07853935
Start iteration #7	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.09766, Loss: -78.07157679
Start iteration #8	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.10547, Loss: -112.62013875
Start iteration #9	. . . . . . . . . . . . . . . . . . . . 	Accuracy: 0.10840, Loss: -363.42530582
Start iteration 

In [191]:
test = [word2index[word] for word in [u'путин']]
last_index = index = test[-1]
print 'GENRATED TEXT:'
for i in range(20):
    pred = predict(test, weights, patterns,unique_words)
    indicies = pred.argsort()[::-1][:20]
    for index in indicies:
        if index in test:
            continue
        else:
            break
    last_index = int(index)
    print index2word[index],
    test.append(index)
    if len(test) > context_len:
        del test[0]

GENRATED TEXT:
украинским комиссии гагаузии помощи был и должен это решение по реформы 0000 в российский магнитского должен проверить стать о еще


# Идеи по улучшению
* первое, что приходит на ум - это увеличить кол-во обучающей выборки
* использовать в качестве контекста, не слова а символы с определнным окном(context_len) равным 40 или больше
* использовать лематизацию или стемминг для словарных "фич", а затем скомбинировать с предыдущим пунктом(пока точно не представляю как)
* модель работает немного медленно, а на больших текстах очень медленно. поэтому можно попробовать искать оптимальные параметры обучения. также можно переписать решение на С/С++ или на Сython

# Использованная литература:
* Tsuruoka Y., Tsujii J., Ananiadou S. Stochastic gradient descent training for l1-regularized log-linear models with cumulative penalty //Proceedings of the Joint Conference of the 47th Annual Meeting of the ACL and the 4th International Joint Conference on Natural Language Processing of the AFNLP: Volume 1-Volume 1. – Association for Computational Linguistics, 2009. – С. 477-485.
* Smith N. A., Eisner J. Contrastive estimation: Training log-linear models on unlabeled data //Proceedings of the 43rd Annual Meeting on Association for Computational Linguistics. – Association for Computational Linguistics, 2005. – С. 354-362.
* Smith N. A. Log-Linear Models // Revised version of thesis research proposal, 2004